In [2]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np

In [11]:
with open("../HP_catalogue.csv") as h:
    lines = h.readlines()

In [12]:
shiplist_raw = [line.split(",")[6] for line in lines[1:] if "/" in line.split(",")[6]]

In [13]:
from collections import Counter, defaultdict

In [14]:
shiplist = sorted(Counter(shiplist_raw).items(), key=lambda x:x[1], reverse=True)

In [16]:
char_in_pair_freq = defaultdict(int)
for pair, score in shiplist:
    left_name, right_name = pair.split("/")
    char_in_pair_freq[left_name] += score
    char_in_pair_freq[right_name] += score
sorted_char_in_pairs = sorted(Counter(char_in_pair_freq).items(), key=lambda x:x[1], reverse=True)

In [17]:
print("Nechme si jen jména s frekvencí větší než 49, zbyde nám jmen:", len([(char, freq) for char, freq in sorted_char_in_pairs if freq >= 49]))


Nechme si jen jména s frekvencí větší než 49, zbyde nám jmen: 101


In [18]:
filtered_char_in_ship_103_best = [(char, freq) for char, freq in sorted_char_in_pairs if freq >= 49]

In [19]:
empty = np.zeros((len(filtered_char_in_ship_103_best), len(filtered_char_in_ship_103_best)), dtype=int)

In [22]:
filtered_shiplist = []
filtered_char_without_freq = [ship for ship, _ in filtered_char_in_ship_103_best]

def index(name):
    """Harry je na nulté pozici, když mu zadám Harry, vrátí 0."""
    return filtered_char_without_freq.index(name)

for ship, count in shiplist:
    left_name, right_name = ship.split("/")
    if left_name in filtered_char_without_freq and right_name in filtered_char_without_freq:
        filtered_shiplist.append((ship, count))
        if left_name == right_name:
            print("Sebevztah", left_name, count)
            empty[index(left_name), index(right_name)] = count
        else:
            empty[index(left_name), index(right_name)] = count
            empty[index(right_name), index(left_name)] = count
        
        
print("Zbývá nám", len(filtered_shiplist), "vztahů.")

Sebevztah OC 20
Sebevztah Original Male Character 11
Sebevztah Original Female Character 9
Sebevztah Original Character(s) 5
Sebevztah Original Female Character(s) 2
Sebevztah Hermione Granger 2
Sebevztah Albus Dumbledore 2
Sebevztah Gellert Grindelwald 2
Sebevztah Harry Potter 2
Sebevztah Tom Riddle 1
Sebevztah Blaise Zabini 1
Sebevztah Pansy Parkinson 1
Zbývá nám 1102 vztahů.


In [23]:
from pandas import DataFrame

In [24]:
df = DataFrame(empty)

keep = np.triu(np.ones(df.shape)).astype('bool').reshape(df.size)


In [25]:
body = []
beautify_chars = [name.replace("_", "&nbsp;") for name in filtered_char_without_freq]
for row_number, name in enumerate(beautify_chars):
    start = f"<tr>"
    start += f"<td>{name}</td>"
    for col_number in range(len(beautify_chars)):
        value = empty[row_number, col_number]
        if col_number >= row_number:
            # obarvení podle hodnoty
            if value >= 500:
                start += f"<td style='background-color: #caffc4; width: 4ex'>{value}</td>"
            elif value >= 100:
                start += f"<td style='background-color: #c2e3bf; width: 4ex'>{value}</td>"
            elif value == 0:
                start += f"<td style='background-color: white; width: 4ex'>{value}</td>"
            else:
                start += f"<td style='background-color: #829487; width: 4ex'>{value}</td>"
        else:
            start += "<td></td>"
    start += "</tr>\n"
    body.append(start)

html_table = f"""
<table>
    <thead style="writing-mode: vertical-lr; text-orientation: use-glyph-orientation; text-align: right">
        <th></th><th>{"</th><th>".join(beautify_chars)}</th>
    </thead>
    {"".join(body)}
</table>
"""


In [26]:
HTML(html_table)

Harry Potter,2,9083,850,186,1620,47,6,24,1024,1107,20,8,240,51,73,5,16,167,10,71,20,93,1,0,1,8,79,458,321,12,66,21,15,13,22,90,2,39,54,37,10,12,34,14,56,6,14,95,77,5,5,3,39,0,21,7,12,5,0,12,3,7,12,15,14,0,18,5,14,32,2,0,4,6,0,4,26,10,5,28,0,1,31,7,1,4,0,11,6,6,0,3,1,1,9,12,0,1,0,2,1
Draco Malfoy,,0,2658,8,63,5,0,4,146,11,0,0,161,0,134,1,15,98,25,25,111,53,0,0,0,4,80,1,4,18,51,2,1,0,2,29,18,3,29,4,6,0,2,7,19,157,1,5,4,3,2,3,0,0,5,2,10,2,0,4,0,0,8,0,3,0,1,0,4,4,0,0,2,0,0,1,7,0,1,18,0,0,0,0,1,3,0,0,6,0,0,0,0,0,0,1,0,0,0,0,0
Hermione Granger,,,2,175,875,88,5,32,78,358,2,0,599,1,15,3,3,89,0,147,4,199,0,0,0,312,48,41,23,85,90,21,191,18,11,35,0,4,92,41,20,39,76,16,10,4,34,23,13,0,2,6,28,0,3,7,77,10,0,0,4,0,3,0,37,0,7,8,31,24,0,0,0,1,0,17,21,2,8,0,0,1,2,5,0,5,0,0,20,0,0,0,7,3,14,3,0,0,0,11,1
Sirius Black,,,,0,138,3142,0,268,9,2,33,0,1,0,120,0,0,3,0,25,97,2,0,0,0,14,0,0,0,11,4,119,0,0,3,0,1,3,0,0,0,12,1,0,4,0,0,0,0,0,0,0,2,0,20,0,0,1,0,0,4,2,5,4,0,0,0,3,2,0,0,50,6,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,5,0,0,0,0,1,0,0,2,0,0
Severus Snape,,,,,0,182,0,104,13,21,232,0,17,37,190,7,0,12,0,97,71,1,0,0,0,4,12,8,18,20,3,29,0,0,7,0,0,1,0,0,1,53,1,0,9,0,5,1,1,0,0,0,0,0,19,17,0,0,0,3,4,1,4,15,0,0,0,1,0,2,0,0,3,0,0,1,1,4,40,15,0,0,2,1,0,1,0,1,0,0,12,0,0,0,0,9,0,0,1,0,0
Remus Lupin,,,,,,0,0,21,3,0,20,0,1,1,62,0,0,3,0,13,64,0,0,0,0,3,7,0,0,9,0,17,0,0,218,0,2,3,0,0,0,6,0,0,4,0,3,0,0,0,0,0,0,0,3,0,0,0,0,7,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,8,0,1,1,0,0,0,1,0,2,0,0,0,0,4,1,0,0,0,0,0,0,0,0,0
Newt Scamander,,,,,,,0,0,0,2,0,556,1,59,15,64,0,1,0,0,55,0,93,576,454,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,6,3,0,0,0,45,0,0,0,0,0,0,29,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,2,0,0,0
James Potter,,,,,,,,0,1,4,1326,0,0,0,23,0,0,0,0,9,11,0,0,0,0,2,0,2,1,3,0,43,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0
Ginny Weasley,,,,,,,,,0,27,0,0,19,0,4,0,1,265,0,9,2,84,0,0,0,0,20,1,0,1,1,0,1,2,7,33,0,0,7,2,5,0,8,0,1,6,2,0,1,0,4,0,2,1,1,5,2,4,0,2,0,1,0,0,1,0,15,0,1,2,0,0,0,1,0,0,0,1,0,0,0,2,1,0,0,6,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
Tom Riddle,,,,,,,,,,1,3,0,3,8,78,3,0,10,0,4,15,3,1,0,0,5,0,0,5,1,0,2,0,0,0,0,0,0,0,0,1,6,0,1,7,0,0,1,1,0,0,0,0,0,5,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,7,0,0,0,0,0,0,0,37,0,1,3,0,2,0,1,0
Lily Evans Potter,,,,,,,,,,,0,0,0,0,1,0,0,2,0,8,0,0,0,0,0,3,0,2,1,36,0,11,0,0,0,0,0,0,0,0,0,1,0,0,4,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,1,0,0,25,0,0,0,0,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0


In [27]:
frame = pd.DataFrame(data=empty, index=filtered_char_without_freq, columns=filtered_char_without_freq)

In [28]:
frame

,Harry Potter,Draco Malfoy,Hermione Granger,Sirius Black,Severus Snape,Remus Lupin,Newt Scamander,James Potter,Ginny Weasley,Tom Riddle,...,Original Male Character,Abraxas Malfoy,John Watson,Dean Winchester,Rubeus Hagrid,Percival Graves | Gellert Grindelwald,Nagini,Alice Longbottom,Padma Patil,Crowley (Good Omens)
Harry Potter,2,9083,850,186,1620,47,6,24,1024,1107,...,3,1,1,9,12,0,1,0,2,1
Draco Malfoy,9083,0,2658,8,63,5,0,4,146,11,...,0,0,0,0,1,0,0,0,0,0
Hermione Granger,850,2658,2,175,875,88,5,32,78,358,...,0,7,3,14,3,0,0,0,11,1
Sirius Black,186,8,175,0,138,3142,0,268,9,2,...,0,0,0,0,1,0,0,2,0,0
Severus Snape,1620,63,875,138,0,182,0,104,13,21,...,0,0,0,0,9,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Percival Graves | Gellert Grindelwald,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Nagini,1,0,0,0,0,0,2,0,1,2,...,0,0,0,0,0,0,0,0,0,0
Alice Longbottom,0,0,0,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Padma Patil,2,0,11,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [33]:
pd.DataFrame(data=empty, 
             index=[i.replace(" ", "_") for i in filtered_char_without_freq], 
             columns=[i.replace(" ", "_") for i in filtered_char_without_freq]
            ).to_csv("../lovers_matrix.csv")

In [ ]:
print("")

In [34]:
[i for i, _ in filtered_shiplist if "Harry" in i]

['Draco Malfoy/Harry Potter',
 'Harry Potter/Severus Snape',
 'Harry Potter/Tom Riddle',
 'Ginny Weasley/Harry Potter',
 'Harry Potter/Hermione Granger',
 'Harry Potter/Tom Riddle | Voldemort',
 'Harry Potter/Voldemort',
 'Harry Potter/Ron Weasley',
 'Harry Potter/Sirius Black',
 'Harry Potter/Luna Lovegood',
 'Cedric Diggory/Harry Potter',
 'Harry Potter/Pansy Parkinson',
 'Blaise Zabini/Harry Potter',
 'Harry Potter/Neville Longbottom',
 'Daphne Greengrass/Harry Potter',
 'Harry Potter/Original Female Character(s)',
 'Harry Potter/Lucius Malfoy',
 'Charlie Weasley/Harry Potter',
 'Harry Potter/Original Male Character(s)',
 'Harry Potter/Theodore Nott',
 'Albus Dumbledore/Harry Potter',
 'Harry Potter/Remus Lupin',
 'Harry Potter/Tony Stark',
 'Harry Potter/Teddy Lupin',
 'George Weasley/Harry Potter',
 'Fleur Delacour/Harry Potter',
 'Harry Potter/Loki (Marvel)',
 'Dudley Dursley/Harry Potter',
 'Harry Potter/Other(s)',
 'Fenrir Greyback/Harry Potter',
 'Harry Potter/James Potter',
 